# trajectory_analyze

particle trajectory analysis

file requirements:
- particle netcdf file generated by parcels after simulation
- the actual ocean data the particles ran on

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import timedelta, datetime
import math
import cartopy.crs as ccrs
import cartopy
import xarray as xr
import numpy as np
from parcels import FieldSet, ParticleSet, JITParticle
import matplotlib.pyplot as plt

from utils import conv_to_dataarray, generate_mask, euc_dist, haversine
from parcels_utils import get_file_info, xr_dataset_to_fieldset
from plot_utils import plot_particles, get_carree_axis

In [ ]:
# lOLOLOLOLOLOL
# the good ol' click coordinates on google maps
coastline_points = np.array([
    [32.527430, -117.123895],
    [32.534549, -117.123832],
    [32.549035, -117.125920],
    [32.556846, -117.130490],
    [32.560474, -117.131934],
    [32.562632, -117.132144],
    [32.564511, -117.132933],
    [32.575317, -117.132941],
    [32.576679, -117.133227],
    [32.593312, -117.133008],
    [32.616745, -117.136719],
    [32.640275, -117.146353],
    [32.662540, -117.160633],
    [32.679572, -117.180263],
    [32.687357, -117.198503],
    [32.683428, -117.223487]
])

tijuana_mouth = np.array([32.551967, -117.127208])

In [ ]:
def line_seg(p1, p2, invert=True):
    if invert:
        p1 = (p1[1], p1[0])
        p2 = (p2[1], p2[0])
    return dict(
        p1=p1,
        p2=p2,
        dom=(p1[0], p2[0]) if p1[0] < p2[0] else (p2[0], p1[0]),
        rng=(p1[1], p2[1]) if p1[1] < p2[1] else (p2[1], p1[1]),
        slope=(p1[1] - p2[1]) / (p1[0] - p2[0])
    )


def valid_point(p, line, invert=True):
    if invert:
        p = (p[1], p[0])
    in_dom = p[0] >= line["dom"][0] and p[0] <= line["dom"][1]
    in_range = p[1] >= line["rng"][0] and p[1] <= line["rng"][1]
    return in_dom and in_range


def intersection_info(p, line, invert=True):
    if invert:
        p = (p[1], p[0])
    norm_slope = -1 / line["slope"]
    slope_d = norm_slope - line["slope"]
    int_d = (line["slope"] * -line["p1"][0] + line["p1"][1]) - (norm_slope * -p[0] + p[1])
    x_int = int_d / slope_d
    y_int = norm_slope * (x_int - p[0]) + p[1]
    return (x_int, y_int), haversine(p[1], y_int, p[0], x_int)

In [ ]:
coast_segs = np.empty(len(coastline_points) - 1, dtype=dict)
for i in range(0, len(coastline_points) - 1):
    coast_segs[i] = line_seg(coastline_points[i], coastline_points[i + 1])

In [ ]:
test_p = (32.575, -117.157062)
# test_p = (32.551228, -117.22)
lat_int = [test_p[0]]
lon_int = [test_p[1]]
for l in coast_segs:
    inted, dist = intersection_info(test_p, l)
    valid = valid_point(inted, l, invert=False)
    if valid:
        lat_int.append(inted[1])
        lon_int.append(inted[0])
    print(inted, dist, valid)

In [ ]:
# ocean current netcdf data the particles used
d_nc = "current_netcdfs/west_coast_1km_hourly/tijuana_river_lin.nc"
# the data generated by ParticleFile
p_nc = "particledata/particle_tijuana_lin.nc"

In [ ]:
d_info = get_file_info(d_nc, 1)
d_ds = d_info["xrds"]
p_ds = xr.open_dataset(p_nc)
fs = xr_dataset_to_fieldset(d_ds)
d_ds.close()
p_ds.close()

In [ ]:
plot_particles(lat_int, lon_int, None, d_info["domain"], show=False)
plt.plot(coastline_points.T[1], coastline_points.T[0])
plt.show()

In [ ]:
ax = get_carree_axis(d_info["domain"])
# plt.scatter(coastline_points.T[1], coastline_points.T[0])
plt.plot(coastline_points.T[1], coastline_points.T[0])

In [ ]:
dists = np.empty((p_ds.dims["traj"], p_ds.dims["obs"]))
for i in range(dists.shape[0]):
    for j in range(dists.shape[1]):
        if np.isnan(p_ds["trajectory"].isel(traj=i, obs=j)):
            dists[i, j:] = np.nan
            break
        lat = float(p_ds["lat"].isel(traj=i, obs=j))
        lon = float(p_ds["lon"].isel(traj=i, obs=j))
        p = (lat, lon)
        least_dist = -1
        for seg in coast_segs:
            inted, dist = intersection_info(p, seg)
            valid = valid_point(inted, seg, invert=False)
            if valid:
                if least_dist == -1 or dist < least_dist:
                    least_dist = dist
        if least_dist == -1:
            for pnt in coastline_points:
                dist = haversine(p[0], pnt[0], p[1], pnt[1])
                if least_dist == -1 or dist < least_dist:
                    least_dist = dist
        dists[i, j] = least_dist

In [ ]:
dists

In [ ]:
d_mask = conv_to_dataarray(generate_mask(d_ds["u"].values), d_ds["u"])

### check whether a particle collided with land

whether it's within the defined distance with the coastline (in meters)

OR

if the particle is in a coordinate without data and has 0 velocity

In [ ]:
def time_to_str(time):
    """
    haha time go chop chop
    
    Args:
        time (np.datetime64)
    """
    return str(time).split(".")[0]

### some immediate problems

- particles are out of bounds even though I use coordinate values on the SAME FIELDSET during simulation
    - ??????????????? how


In [ ]:
collide_dist_thresh = 100  # meters
collided_land = np.zeros(p_ds["trajectory"].shape, dtype=bool)
lat_ls_coll = []
lon_ls_coll = []
lat_ls_broke = []
lon_ls_broke = []
times_sec = []
times_dates = []
part_dists = []
for p_iter in range(len(p_ds["trajectory"])):
    p = p_ds["trajectory"][p_iter]
    # find the first state of the particle where it got stuck on land something
    for step in range(len(p)):
        if np.isnan(p[step]):
            break
        time = p_ds["time"].values[p_iter, step]
        time_secs = p_ds["lifetime"].values[p_iter, step] + p_ds["spawntime"].values[p_iter, step]
        lat = p_ds["lat"].values[p_iter, step]
        lon = p_ds["lon"].values[p_iter, step]
        times_sec.append(time_secs)
        times_dates.append(time)
        part_dists.append(dists[p_iter, step])
        # first, check if the vector speed at that coordinate is 0 after interpolation
        # second, check if the particle is not in an invalid coordinate (a coordinate supposed to have data)
        total_vel = abs(fs.U[time_secs, 0, lat, lon]) + abs(fs.V[time_secs, 0, lat, lon])
        inland = total_vel <= 0 and not d_mask.sel(time=time, lat=lat, lon=lon, method="nearest")
        near_coast = dists[p_iter, step] <= collide_dist_thresh and dists[p_iter, step] >= 0
        if inland or near_coast:
            lat_ls_coll.append(lat)
            lon_ls_coll.append(lon)
            print(f"particle {p_iter} beached at time {time_to_str(time)}")
            print(f"    spawn time: {time_to_str(p_ds['time'].values[p_iter, 0])}")
            print(f"    spawn coord: ({p_ds['lat'].values[p_iter, 0]}, {p_ds['lon'].values[p_iter, 0]})")
            print(f"    lifetime: {p_ds['lifetime'].values[p_iter, step]}")
            break

30, 31, 32, 33, 34, 37, 38, 39

In [ ]:
plot_particles(lat_ls_coll, lon_ls_coll, None, d_info["domain"])